In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import numpy as np
import pickle
import spacy
from collections import defaultdict
import numpy as np

In [2]:
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'cocktails',    # DB that we are connecting to
    'port': 5432        
}


connection = pg.connect(**connection_args)

In [3]:

query = "SELECT * FROM drinks;"

drink_df = pd_sql.read_sql(query, connection)


In [5]:
# Create list of ingredients and names

names = []
ingredient_list = []
for jj in range(0, len(drink_df)):
    ingredients = []
    if drink_df.at[jj, 'stralcoholic'] == 'Alcoholic' and drink_df.at[jj, 'strcategory'] != 'Homemade Liqueur':
        for ii in range(1,16):
            ingredient_column = 'stringredient' + str(ii)
            if isinstance(drink_df.at[jj, ingredient_column], str):
                ingredients.append(drink_df.at[jj, ingredient_column].lower())
        names.append(drink_df.at[jj,'strdrink'])
        ingredient_list.append(' '.join(ingredients))


In [6]:
# Read the giant spacy english language file

nlp = spacy.load('en_core_web_lg')

In [40]:
# Create dictionary with top twenty spacy similarity matches

match_dictionary_full = defaultdict(list)
for jj in range(0, len(ingredient_list)):
    best_scores = np.zeros(20)
    best_cocktail = np.empty(20, dtype=object)                
    for ii in range(0,len(ingredient_list)):
        if jj == ii:
            continue        
        cocktail1_tokens = nlp(ingredient_list[jj])
        cocktail2_tokens = nlp(ingredient_list[ii])
        similarity = cocktail1_tokens.similarity(cocktail2_tokens)
        if similarity > np.min(best_scores):
            change = np.argmin(best_scores)
            best_cocktail[change] = names[ii]
            best_scores[change] = similarity
    match_dictionary_full[names[jj].lower()] = best_cocktail.tolist()

In [39]:
# Output to file for use with app

with open('data/cocktail_recommender.pickle', 'wb') as fp:
    pickle.dump(match_dictionary_full, fp)